### requirements

numpy==2.0.2
pandas==2.3.0
scikit-learn==1.7.0
catboost==1.2.8
joblib==1.5.1
sentence-transformers==5.1.1
transformers==4.53.0
tokenizers==0.21.2
torch==2.6.0
scipy==1.15.3

In [2]:
!pip install -q -U sentence-transformers > /dev/null 2>&1

In [1]:
!pip install -q catboost > /dev/null 2>&1

In [ ]:
import gc
import os
import re
import warnings

from catboost import CatBoostRanker, Pool
import joblib
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [21]:
data = pd.read_parquet('train-dset.parquet')

In [22]:
data.head()

,query_id,item_id,query_text,item_title,item_description,query_cat,query_mcat,query_loc,item_cat_id,item_mcat_id,item_loc,price,item_query_click_conv,item_contact
0,4,7349717282,ботинки детские zara 21,Ботинки детские Zara,Новые полуботинки фирмы Zara. \nразмеры 21 сте...,29.0,38.0,624480.0,29,2179540,638660,500.0,-1.0,0.0
1,4,7519735286,ботинки детские zara 21,Детские ботинки Zara унисекс,"Крутые ботинки, в отличном состоянии",29.0,38.0,624480.0,29,2179540,637640,250.0,-1.0,0.0
2,4,4384449104,ботинки детские zara 21,Ботинки детские zara,Челси димесезонные Zara \nВ идеальном состояни...,29.0,38.0,624480.0,29,2179540,623880,1500.0,-1.0,0.0
3,4,7283365509,ботинки детские zara 21,Детские ботиночки Zara 21 размер,АВИТО ДОСТАВКА .21 РАЗМЕР.,29.0,38.0,624480.0,29,2179540,628530,220.0,-1.0,0.0
4,4,4452768560,ботинки детские zara 21,Детские ботиночки zara размер 21,Детские ботинки Zara \nРазмер 21 - 13 см\nСост...,29.0,38.0,624480.0,29,2179540,637640,1648.0,-1.0,1.0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7781790 entries, 0 to 7781789
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   query_id               int64  
 1   item_id                int64  
 2   query_text             object 
 3   item_title             object 
 4   item_description       object 
 5   query_cat              float32
 6   query_mcat             float64
 7   query_loc              float32
 8   item_cat_id            int32  
 9   item_mcat_id           int32  
 10  item_loc               int32  
 11  price                  float32
 12  item_query_click_conv  float32
 13  item_contact           float32
dtypes: float32(5), float64(1), int32(3), int64(2), object(3)
memory usage: 593.7+ MB


In [25]:
data.isna().sum().sort_values(ascending=False)

query_mcat               1761233
item_description             107
item_title                   107
item_id                        0
query_text                     0
query_id                       0
query_cat                      0
query_loc                      0
item_cat_id                    0
item_mcat_id                   0
item_loc                       0
price                          0
item_query_click_conv          0
item_contact                   0
dtype: int64

query_mcat я использовать не планирую, так как query_mcat и item_mcat_id сильно отличаются, а в item_description и item_title заполню пропуски перед нормализацией.

In [26]:
data.item_query_click_conv.value_counts()

item_query_click_conv
-1.000    6578159
 0.000     636977
 0.333      23477
 0.250      21156
 0.200      19078
           ...   
 0.870          1
 0.459          1
 0.452          1
 0.427          1
 0.369          1
Name: count, Length: 557, dtype: int64

#### Разбор признаков:

- query_id - идентификатор запроса. Скорее всего все запросы с одинаковым индексом принадлежат одному пользователю, так как item_contact в большинстве случаев = 0.
- item_id - идентификатор товара.
- query_text, item_title, item_description - текстовые данные.
- query_cat, query_mcat - практически не отличаются. Скорее всего одно - это большая категория, а второе - подкатегория.
- query_loc - локация запроса (город / район).
- item_cat_id, item_mcat_id - тоже что и с категориями запросов.
- item_loc - локация товара (город / район).
- price - цена товара.
- item_query_click_conv - у большей части товаров конверсия -1 или 0.
- item_contact - взаимодействие пользователя с товаром (покупка или открытие карточки). Для некоторых пар запрос-товар нет ни одного взаимодействия.

## Идея решения

Изначальные признаки не информативны и нужно на их основе посчитать новые. Имея новые признаки использую модель CatBoostRanker для реранкинга.

Новые признаки:

- Нормализованное отклонение цены от медианы внутри группы. Это даст модели понимание того какие товары дешевле, а какие дороже для каждой группы.
- Нормализованная разница между парами: query_cat и item_cat_id, query_loc и item_loc. Этот признак нкжен на случай если похожие категории находятся рядом по их идентификаторам.
- Равенство между парами: query_cat и item_cat_id, query_loc и item_loc. Этот признак в отчие от предыдущего явно указывает на скодсво категории.
- Косинусное сходство по TF-IDF между парами (query_text, item_title) и (query_text, item_description). Это покажет схожесть текста запроса с заголовком товара и его описанием.
- Косинусное сходство между эмбеддингами пар (query_text, item_title) и (query_text, item_description). Это покажет схожесть смыслов в парах.

## Предобработка данных

In [ ]:
def normalize_text(s: str) -> str:
    '''
    Функция для нормализации текста.
    Приводит текст к нижнему регистру и оставляет только буквы и цифры.
    '''
    
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-zа-яё0-9]+", " ", s)  
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [ ]:
def preprocessing(df: pd.DataFrame) -> None:
    '''
    Функция для предобработки данных.
    Вычисления внутри функции:
    - отклонение цены от медианы внутри группы и нормализация этого отклонения
    - разница между парами значений (query_cat, item_cat_id), (query_loc, item_loc)
    - сходство между парами значений (query_cat, item_cat_id), (query_loc, item_loc)
    - нормализация текстовых данных и сокращение длины текста в столбце item_description
    '''
    
    # Отклонения цены от медианной внутри группы
    df["median_price"] = df.groupby("query_id")["price"].transform("median")
    df['price_median_difference'] = df.price - df.median_price
    # Подсчет стандартного отклонения внутри группы
    std_g = (df.groupby('query_id')['price_median_difference']
                .transform('std')
                .replace(0, np.nan)
                .fillna(1)
            )
    # Нормализация
    df['price_median_difference_std'] = df['price_median_difference'] / std_g
    print('price_median_difference_std done')

    # Вычисление разницы между парами: query_cat и item_cat_id, query_loc и item_loc
    df['cat_difference'] = df.query_cat - df.item_cat_id
    print('cat_difference done')
    df['loc_difference'] = df.query_loc - df.item_loc
    print('loc_difference done')

    # Вычисление равенства между парами: query_cat и item_cat_id, query_loc и item_loc
    df['cat_equality'] = pd.Series(df.query_cat == df.item_cat_id).astype('int')
    print('cat_equality done')
    df['loc_equality'] = pd.Series(df.query_loc == df.item_loc).astype('int')
    print('loc_equality done')

    # Нормализация текста
    df['query_text_norm'] = df['query_text'].fillna("").map(normalize_text)
    print('query_text_norm done')
    df['item_title_norm'] = df['item_title'].fillna("").map(normalize_text)
    print('item_title_norm done')
    df['item_description_norm'] = df['item_description'].fillna("").map(normalize_text)
    # Сокращу item_description_norm для ускорения обучения
    df["item_description_norm"] = df["item_description_norm"].str.slice(0, 100)
    print('item_description_norm done')

### Предобработка тренировочной выборки

In [ ]:
preprocessing(data)

In [ ]:
# Нормализация признаков
scaler = StandardScaler()

data.loc[:, ['cat_difference', 'loc_difference']] = scaler.fit_transform(data[['cat_difference', 'loc_difference']])  

In [ ]:
# Создание словаря для TF-IDF
all_words = pd.concat([data["query_text_norm"], data["item_title_norm"], data["item_description_norm"]], axis=0)

In [ ]:
# Обучение TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1,2), # использую униграммы и биграммы
    min_df=2, # фильтрация от редких токенов
    max_df=0.95, # фильтрация от частых токенов
    sublinear_tf=True 
)
tfidf.fit(all_words)

In [ ]:
Q  = tfidf.transform(data["query_text_norm"]) # матрица запросов
T  = tfidf.transform(data["item_title_norm"]) # матрица заголовков
D = tfidf.transform(data["item_description_norm"]) # матрица описаний

In [ ]:
# Расчет косинусного сходства для пар (запрос, заголовок), (запрос, описание)
data["tfidf_cos_query_title"] = (Q.multiply(T)).sum(axis=1).A1
data["tfidf_cos_query_desc"] = (Q.multiply(D)).sum(axis=1).A1

In [ ]:
# Удаление признаков, которые не будут использованы при обучении
data.drop(columns=['query_text', 'item_title', 'item_description', 'query_cat', 'query_mcat', 'query_loc', 'item_cat_id',
                  'item_mcat_id', 'item_loc', 'price', 'median_price', 'price_median_difference'], inplace=True)

In [ ]:
data.head()

In [ ]:
# Сохранение данных
data.to_parquet('train-dset-prep.parquet')

In [ ]:
# Сохранение обученных StandardScaler и TfidfVectorizer
joblib.dump(scaler, "standard_scler.joblib")
joblib.dump(tfidf, "tfidf_vectorizer.joblib")

### Предобработка тестовой выборки

Использую те же преобразования что и для тренировочной выборки

In [ ]:
test_data = pd.read_parquet('test-dset-small.parquet')

In [ ]:
preprocessing(test_data)

In [ ]:
# При необходимости
scaler = joblib.load("standard_scler.joblib")
tfidf = joblib.load("tfidf_vectorizer.joblib")

In [ ]:
test_data.loc[:, ['cat_difference', 'loc_difference']] = scaler.transform(test_data[['cat_difference', 'loc_difference']])

In [ ]:
Q_t  = tfidf.transform(test_data["query_text_norm"]) # матрица запросов
T_t  = tfidf.transform(test_data["item_title_norm"]) # матрица заголовков
D_t = tfidf.transform(test_data["item_description_norm"]) # матрица описаний

In [ ]:
test_data["tfidf_cos_query_title"] = (Q_t.multiply(T_t)).sum(axis=1).A1
test_data["tfidf_cos_query_desc"] = (Q_t.multiply(D_t)).sum(axis=1).A1

In [ ]:
test_data.drop(columns=['query_text', 'item_title', 'item_description', 'query_cat', 'query_mcat', 'query_loc', 'item_cat_id',
                  'item_mcat_id', 'item_loc', 'price', 'median_price', 'price_median_difference'], inplace=True)

In [ ]:
test_data.head()

In [ ]:
test_data.to_parquet('test-dset-prep.parquet')

## Расчет косинусного сходства между эмбеддингами

На этом этапе я использовал accelerator - GPU T4 x 2 на платформе Kaggle

In [ ]:
# Загрузка данных
data = pd.read_parquet('train-dset-prep.parquet')

In [ ]:
# Разбиение тренировочной выборки
train_data = data.iloc[:5_000_000, :]
valid_data = data.iloc[5_000_000:, :]

In [ ]:
def compute_embedding_cosines(df: pd.DataFrame) -> None:
    '''Функция для вычисления косинусного сходства между эмбеддингами'''
    
    os.environ["TOKENIZERS_PARALLELISM"] = "true" # разрешаю многопоточную токенизацию HuggingFace
    
    MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    
    model = SentenceTransformer(MODEL, device="cpu") # загрузка модели на CPU, дальше она перенесется на GPU
    model.max_seq_length = 256 # ограничение максимальной длины последовательности

    pool = model.start_multi_process_pool(target_devices=["cuda:0", "cuda:1"]) # копирование модели на оба GPU
    
    BATCH = 1024 # размер мини-батча на каждом GPU
    CHUNK = 2000 # количество строк в чанке
    STEP  = 100_000 # количество строк для одного шага обработки
    
    N = len(df) # количество строк в наборе данных
    # массивы для сохраниния косинусного сходства
    cos_q_t = np.empty(N, dtype=np.float32)
    cos_q_d = np.empty(N, dtype=np.float32)
    # списки с текстами для кодирования моделью
    q_list  = df["query_text_norm"].fillna("").tolist()
    t_list  = df["item_title_norm"].fillna("").tolist()
    d_list  = df["item_description_norm"].fillna("").tolist()
    
    def as_2d(a):
        '''Вспомогательная функция для приведения массивов к одинаковому размеру'''
        a = np.asarray(a)
        return a[None, :] if a.ndim == 1 else a
    
    i = 0 # нужно только для визуализации прогресса
    for start in range(0, N, STEP): # главный цикл: идём по датафрейму окнами размера STEP, чтобы не переполнять память
        print(f'step {i}')
        i += 1

        end = min(N, start + STEP) # граница текущего окна
        # беру срезы данных соответствующие размерам окна
        q_batch = q_list[start:end]
        t_batch = t_list[start:end]
        d_batch = d_list[start:end]
    
        # считаю эмбеддинги. normalize_embeddings=True для L2-нормировки векторов
        q_emb = model.encode_multi_process(q_batch, pool, batch_size=BATCH, chunk_size=CHUNK,
                                           normalize_embeddings=True, show_progress_bar=False)
        t_emb = model.encode_multi_process(t_batch, pool, batch_size=BATCH, chunk_size=CHUNK,
                                           normalize_embeddings=True, show_progress_bar=False)
        d_emb = model.encode_multi_process(d_batch, pool, batch_size=BATCH, chunk_size=CHUNK,
                                           normalize_embeddings=True, show_progress_bar=False)

        # приведение к одному размеру. float32 для экономии памяти
        q_emb = as_2d(q_emb).astype(np.float32, copy=False)
        t_emb = as_2d(t_emb).astype(np.float32, copy=False)
        d_emb = as_2d(d_emb).astype(np.float32, copy=False)
    
        # фактический размер текущего окна (на случай, если он - одна строка)
        L = q_emb.shape[0]
        end_slice = start + L
    
        # подсчет косинусов через скалярное произведение
        cos_q_t[start:end_slice] = np.einsum("ij,ij->i", q_emb, t_emb)
        cos_q_d[start:end_slice] = np.einsum("ij,ij->i", q_emb, d_emb)
    
    # запись нового признака в набор данных
    df["embedding_cos_query_title"] = cos_q_t
    df["embedding_cos_query_description"] = cos_q_d

### Добавление косинусного сходства между эмбеддингами в train

In [ ]:
compute_embedding_cosines(train_data)

In [ ]:
train_data.head()

In [ ]:
train_data.to_parquet('train-dset-prep-full.parquet')

### Добавление косинусного сходства между эмбеддингами в valid

In [ ]:
compute_embedding_cosines(valid_data)

In [ ]:
valid_data.head()

In [ ]:
valid_data.to_parquet('valid-dset-prep-full.parquet')

### Добавление косинусного сходства между эмбеддингами в test

In [ ]:
test_data = pd.read_parquet('test-dset-prep.parquet')

In [ ]:
compute_embedding_cosines(test_data)

In [ ]:
test_data.head()

In [ ]:
test_data.to_parquet('test-dset-prep-full.parquet')

## Обучение модели реранкинга

In [3]:
train_data = pd.read_parquet('train-dset-prep-full.parquet')
valid_data = pd.read_parquet('valid-dset-prep-full.parquet')

In [4]:
# Сортировка данных по идентификатору запроса
train_data = train_data.sort_values('query_id').reset_index(drop=True)
valid_data = valid_data.sort_values('query_id').reset_index(drop=True)

In [5]:
feature_cols = train_data.drop(columns=['item_contact', 'query_text_norm', 'item_title_norm', 'item_description_norm']).columns.tolist()
cat_cols = ['cat_equality', 'loc_equality']
# Создание тренировочного и валидационного пула
train_pool = Pool(train_data[feature_cols], label=train_data['item_contact'], group_id=train_data['query_id'], cat_features=cat_cols)
valid_pool = Pool(valid_data[feature_cols], label=valid_data['item_contact'], group_id=valid_data['query_id'], cat_features=cat_cols)

In [6]:
model = CatBoostRanker(
    loss_function="YetiRank", # listwise-ранжирование
    eval_metric="NDCG:top=10", # метрика
    iterations=1400, # количество итераций
    learning_rate=0.05, # шаг бустинга
    depth=8, # глубина симметричных деревьев
    l2_leaf_reg=7, # L2-регуляризация значений в листьях
    rsm=0.8, # доля признаков на дерево
    subsample=0.9, # доля объектов на дерево
    bootstrap_type="Bernoulli", # независимое включение каждой строки с вероятностью subsample
    border_count=64, # квантизация числовых признаков в дискретные бины (CatBoost так ускоряет обучение на числовых фичах)
    sampling_frequency="PerTreeLevel", # переотбор кандидатов на каждом уровне дерева
    random_seed=42 # ответ на главный вопрос жизни, вселенной и всего такого
)

In [ ]:
# Обучение модели
model.fit(train_pool, eval_set=valid_pool, verbose=200)

In [ ]:
# Сохранение обученной модели
model.save_model("ranker.cbm", format="cbm")

### Реранкинг тестовой выборки

In [8]:
test_data = pd.read_parquet('test-dset-prep-full.parquet')

In [11]:
test_data = test_data.sort_values('query_id').reset_index(drop=True)

feature_cols = test_data.drop(columns=['query_text_norm', 'item_title_norm', 'item_description_norm']).columns.tolist()
cat_cols = ['cat_equality', 'loc_equality']

test_pool = Pool(
    test_data[feature_cols],
    group_id=test_data['query_id'],
    cat_features=cat_cols
)

pred = model.predict(test_pool)
test_data['cb_score'] = pred

test_data = test_data.sort_values(['query_id', 'cb_score'], ascending=[True, False])
submission_df = test_data[['query_id', 'item_id']]

In [ ]:
submission_df.info()

In [ ]:
submission_df.head()

In [20]:
submission_df.to_csv(
    'solution.csv', 
    header=['query_id', 'item_id'], 
    index=False,
)